C# 8 **readonly members**

In [19]:
// original

public struct Worker
{
	int data;

    public void Mutate()
    {
        data = data + 1;
    }
}

new Worker().Mutate();

In [22]:
// readonly intent

public struct Worker
{
	int data;

    public readonly void Mutate()
    {
        data = data + 1; // error
    }
}

new Worker().Mutate();

Error: (9,9): error CS1604: Cannot assign to 'data' because it is read-only

C# 8 **Default interface method**

In [40]:
// original

interface IAnimal
{
	int Age { get; set; }
}

interface IAnimal2 : IAnimal
{
	int Birth();
}

public abstract class Animal : IAnimal2
{
	public int Age { get; set; } = 10;
	public int Birth() => DateTime.Now.Year - Age;
}

public class Fish : Animal { }
var f = new Fish();
f.Birth().Display();

2013

In [42]:
// default methods

interface IAnimal
{
	int Age { get; set; }
	int Birth() => DateTime.Now.Year - Age;
}

public abstract class Animal : IAnimal
{
	public int Age { get; set; } = 10;
}

public class Fish : Animal { }
var f = new Fish() as IAnimal;
f.Birth().Display();

2013

C# 8 **Switch expression**

In [49]:
// switch case statements

enum Vertical { Up, Down }

var direction = Vertical.Down;

char character;

switch (direction)
{
    case Vertical.Up:
        character = '\u2191';
        break;
    case Vertical.Down:
        character = '\u2193';    
        break;
}

Console.WriteLine(character);  // Output: ↓

↓


In [ ]:
// switch expressions

enum Vertical { Up, Down }

var direction = Vertical.Down;

char character = direction switch
{
    Vertical.Up => '\u2191',
    Vertical.Down => '\u2193',
    _ => throw new Exception()
};
Console.WriteLine(character);  // Output: ↓

C# 8 **Property matching**

In [3]:
// original
var day = DateTime.Now;

var today1 = day.Year == 2023 && day.Month == 6;

var today2 = day switch
{
	var d when d.Year == 2023 
		&& d.Month == 6 => "Yes",
	_ => "No"
};

var today3 = false;
if (day.Year == 2023 && day.Month == 6)
{
	today3 = true;
}

Console.WriteLine(today1); // True
Console.WriteLine(today2); // Yes
Console.WriteLine(today3); // False

True
Yes
True


In [4]:
// property matching
var day = DateTime.Now;

var today1 = day is { Year: 2023, Month: 6 };

var today2 = day switch 
{
	{ Year: 2023, Month: 6 } => "Yes",
	_ => "No"
};

var today3 = false;
if (day is { Year: 2023, Month: 6 })
{
	today3 = true;
}

Console.WriteLine(today1); // True
Console.WriteLine(today2); // Yes
Console.WriteLine(today3); // False

True
Yes
False


C# 8 **Positional matching**

In [12]:
// orignial

var t = (1, 2, 3, 4);

if (t.Item1 == 1 && t.Item2 == 2 && t.Item3 != null)
{
    Console.WriteLine("Yes");
}

if (t.Item1 == 1 && t.Item3 == 3)
{
    Console.WriteLine("Yes");
}


Yes
Yes


In [14]:
// Positional matching

var t = (1, 2, 3, 4);

if (t is (1, 2, 3, _))
{
	Console.WriteLine("Yes");
}

if (t is (1, _, 3, _))
{
	Console.WriteLine("Yes");
}

if (t is not (1, _, 3, _))
{
	Console.WriteLine("No");
}

Yes
Yes


C# 8 **Tuple matching**

In [66]:
// original

var t = ("Jeremy", "Waters");

string m = null;

if (t.Item1 == "Tim" && t.Item2 != "Valley")
    m = "Tim = yes, Valley = no";

else if (t.Item1 != "Tim" && t.Item2 == "Valley")
    m = "Tim = no, Valley = yes";

else if (t.Item1 != "Tim" && t.Item2 != "Valley")
    m = "Tim = no, Valley = no";
    
else if (t.Item1 == "Tim" && t.Item2 == "Valley")
    m = "Tim = no, Valley = no";

m.Display(); // Tim = no, Valley = no


Tim = no, Valley = no

In [63]:
// tuple matching

var t = ("Jeremy", "Waters");

var m = t switch
{
	("Tim", _) and not (_, "Valley") 
		=> "Tim = yes, Valley = no",

	not ("Tim", _) and (_, "Valley") 
		=> "Tim = no, Valley = yes",

	not ("Tim", _) and not (_, "Valley") 
		=> "Tim = no, Valley = no",
		
	("Tim", "Valley") => "Tim = no, Valley = no",
};

m.Display(); // First and last

Tim = no, Valley = no

C# 8 **Using directives**

In [71]:
// try/finally
var conn = new Connection();
try { conn.Open(); }
finally { conn.Close(); } 

// using
void Do()
{
	using (var conn = new Connection())
	{
		conn.Open();
	}
}

class Connection : IDisposable
{
	public void Open() { /* TODO */ }
	public void Close() { /* TODO */ }
	public void Dispose() => Close();
}

In [72]:
// try/finally
var conn = new Connection();
try { conn.Open(); }
finally { conn.Close(); } 

// using
void Do()
{
	using var conn2 = new Connection();
	conn2.Open();
}

class Connection : IDisposable
{
	public void Open() { /* TODO */ }
	public void Close() { /* TODO */ }
	public void Dispose() => Close();
}

C# 8 **Static local functions**

In [ ]:
// original

double Divide(double divisor, double dividend)
{
	ValidateDividend(dividend);
	
	return divisor / dividend;

	void ValidateDividend(double dividend)
	{
		if (dividend == 0)
		{
			throw new Exception();
		}
	}
}

In [73]:
// static local method

double Divide(double divisor, double dividend)
{
	ValidateDividend(dividend);
	
	return divisor / dividend;

	static void ValidateDividend(double dividend)
	{
		if (dividend == 0)
		{
			throw new Exception();
		}
	}
}

C# 8 **Disposable ref structs**

In [ ]:
// Error CS8343: ref structs cannot implement interfaces

ref struct Book : IDisposable // error
{
   public void Dispose()
   {
   }
}

In [ ]:
// explicit cleanup (deterministic finalization)

ref struct Book
{
   public void Dispose()
   {
   }
}

public void Do()
{
	using var book = new Book();
	// TODO
}

C# 8 **Nullable reference types**

In [74]:
// original (NullReferenceException)

void Do(string message)
{
	var value = message.ToLower(); // runtime error
	Console.WriteLine(value); 
}

In [ ]:
// original 

#nullable enable

public class User
{
	public string _name; // static warning
}

In [172]:
// set in constructor

#nullable enable

public class User
{
	public string _name;

	public User()
	{
		_name = string.Empty;
	}
}

In [171]:
// original 

#nullable enable

public class User
{
	public string _name;

	public User() // static warning
	{
		DefaultValues();
	}

	public void DefaultValues() 
	{
		_name = string.Empty;
	}
}

In [92]:
// MemberNotNull

#nullable enable

using System.Diagnostics.CodeAnalysis;

public class User
{
	public string _name;

	public User() => DefaultValues();

	[MemberNotNull(nameof(_name))] // static analysis
	public void DefaultValues() 
	{
		_name = string.Empty;
	}
}

In [173]:
// original

#nullable enable

public class User
{
	public string _first = string.Empty;

	public string _middle = null; // static warning
	
	public string _last = string.Empty;
}

In [ ]:
// nullable modifer

#nullable enable

public class User
{
	public string _first = string.Empty;

	public string? _middle = null; 
	
	public string _last = string.Empty;
}

In [174]:
// null-forgiving

#nullable enable

public class User
{
	public string _first = string.Empty;

	public string _middle = null!;
	
	public string _last = string.Empty;
}

In [7]:
// NullableAttribute

using System.Diagnostics.CodeAnalysis;

#nullable enable

public class User
{
#nullable disable
	public string _first = null; // no warning
#nullable enable

	[AllowNull]
	public string _middle = null; // no warning
	
	[NotNull]
	public string _last = string.Empty;
}

C# 8 **Asynchronous streams**

In [108]:
// original

public IEnumerable<int> Get()
{
	foreach (var item in Enumerable.Range(1, 10))
	{
		var x = ProcessAsync(item)
			.GetAwaiter().GetResult();
		yield return x;
	}
}

public async Task<int> ProcessAsync(int item) => ++item;

Console.WriteLine(Get().Sum());

foreach (var item in Get())
{
	Console.WriteLine(item);
}

65
2
3
4
5
6
7
8
9
10
11


In [122]:
// async streams

#r "nuget:System.Linq.Async"
using System.Linq;

public async IAsyncEnumerable<int> GetAsync()
{
	foreach (var item in Enumerable.Range(1, 10))
	{
		yield return await Process(item);
	}
}

public async Task<int> Process(int item) => ++item;

var x = await GetAsync().SumAsync();
Console.WriteLine(x);

await foreach (var item in GetAsync())
{
	Console.WriteLine(item);
}

Installed Packages System.Linq.Async, 6.0.1

65
2
3
4
5
6
7
8
9
10
11


C# 8 **Index & Range**

In [136]:
// original
var a = new[] { "a", "b", "c", "d", "e" };

// first
a[0].Display(); // a

// last
a[a.Length - 1].Display(); // e

// second & third
var b = new ArraySegment<string>(a, 1, 2);
b.Display(); // b, c

// first 4
var c = new string[4];
Array.Copy(a, c, 4);
c.Display(); // a, b, c, d

// last 4
var d = a.Reverse().Take(4).Reverse();
d.Display(); // b, c, d, e

a

e

[ b, c ]

[ a, b, c, d ]

[ b, c, d, e ]

In [139]:
// original
var a = new[] { "a", "b", "c", "d", "e" };

// first
a[0].Display(); // a

// last
a[^1].Display(); // e

// second & third (end is exclusive)
a[1..3].Display(); // b, c

// first 4 (end is exclusive)
a[..4].Display(); // a, b, c, d

// last 4
a[^4..].Display(); // b, c, d, e

a

e

[ b, c ]

[ a, b, c, d ]

[ b, c, d, e ]

C# 8 **null coalescing assignment**

In [196]:
// original

var w = (string)null;
var x = (string)null;
var y = (string)null;
var z = (string)null;

string a = "mouse", b = "cat", c = "dog";

// original

if (x is null) 
	if (y is null) 
		if (z is null) w = a; 
		else w = z; 
	else w = y;
else w = x;

if (x is null) x = (y is null) ? b : y;

if (w is null) 
	if (x is null) 
		if (z is null) y = b; 
		else y = z; 
	else y = x;
else y = w;

if (z is null) z = c;

w.Display(); // Output: mouse
x.Display(); // Output: cat
y.Display(); // Output: mouse
z.Display(); // Output: dog

mouse

cat

mouse

dog

In [197]:
var w = (string)null;
var x = (string)null;
var y = (string)null;
var z = (string)null;

string a = "mouse", b = "cat", c = "dog";

// null coalesce

w = x ?? y ?? z ?? a;
x ??= y ?? b;
y = w ?? x ?? y ?? z ?? b;
z ??= c;

w.Display(); // Output: mouse
x.Display(); // Output: cat
y.Display(); // Output: mouse
z.Display(); // Output: dog


mouse

cat

mouse

dog

In [170]:
#nullable enable
string name = null!;
name = "";
name = null;
name?.GetType()

<null>